# StoryLand AI - Modular Demo

**Transform books into travel adventures using modular AI agents.**

This notebook demonstrates the modular version of StoryLand AI, where agents, tools, and services are organized into reusable packages.

## Setup

In [9]:
# Import libraries
import os
import uuid
import logging
import json
from dotenv import load_dotenv

# Import Google ADK components
from google.genai import types
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner

# Import from our modules
from common.config import load_config
from common.logging import setup_logger

from services.session_service import create_session_service
from services.context_manager import ContextManager

from tools.google_books import google_books_tool
from agents.orchestrator import create_workflow

# Load environment
load_dotenv()

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(name)s - %(message)s'
)

logger = setup_logger("storyland_demo")

print("Environment Setup Complete")
print(f"   Google API Key: {'OK' if os.getenv('GOOGLE_API_KEY') else 'MISSING'}")

Environment Setup Complete
   Google API Key: OK


## Configuration

Configure your book and session settings here:

In [10]:
# ============================================================
# CONFIGURE YOUR BOOK HERE
# ============================================================
BOOK_TITLE = "gone with the wind"
AUTHOR = ""  # Optional - leave empty if unknown

# Session configuration
USE_DATABASE = False  # Set to True to use SQLite for persistent sessions
USER_ID = "user1"

# User preferences (used for personalization)
USER_PREFERENCES = {
    "budget": "moderate",          # budget, moderate, luxury
    "preferred_pace": "moderate",  # relaxed, moderate, fast-paced
    "prefers_museums": True,
    "travels_with_kids": False,
}
# ============================================================

print(f"Book: {BOOK_TITLE}")
print(f"Author: {AUTHOR or 'Unknown'}")
print(f"Database: {'Enabled' if USE_DATABASE else 'Disabled'}")
print(f"Preferences: {USER_PREFERENCES}")

Book: gone with the wind
Author: Unknown
Database: Disabled
Preferences: {'budget': 'moderate', 'preferred_pace': 'moderate', 'prefers_museums': True, 'travels_with_kids': False}


## Initialize Services

Create the model, session service, and workflow:

In [11]:
# Load configuration
config = load_config()

# Configure Gemini model
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504]
)

model = Gemini(
    model=config.model_name,
    api_key=config.google_api_key,
    retry_options=retry_config
)

print(f"Model configured: {model.model}")

# Create session service
session_service = create_session_service(use_database=USE_DATABASE)
print("Session service created")

# Create context manager
context_manager = ContextManager(max_events=20)
print("Context manager created")

# Create workflow
workflow = create_workflow(model, google_books_tool)
print("Workflow created")

# Create runner
runner = Runner(
    agent=workflow,
    app_name="storyland",
    session_service=session_service
)
print("Runner created")

2025-11-22 14:40:44,353 - WARNING - google_adk.google.adk.runners - App name mismatch detected. The runner is configured with app name "storyland", but the root agent was loaded from "/Users/osa/Documents/storyland/storyland-ai/.venv/lib/python3.12/site-packages/google/adk/agents", which implies app name "agents".


Model configured: gemini-2.0-flash-lite
Using InMemorySessionService (not persistent)
Session service created
Context manager created
Workflow created
Runner created


## Create Session

In [12]:
# Create a new session
session_id = str(uuid.uuid4())

await session_service.create_session(
    app_name="storyland",
    user_id=USER_ID,
    session_id=session_id,
    state={
        "book_title": BOOK_TITLE,
        "author": AUTHOR,
        # User preferences - accessed by reader_profile_agent via get_preferences_tool
        "user:preferences": USER_PREFERENCES
    }
)

print(f"Session created: {session_id[:8]}...")
print(f"Preferences stored in state['user:preferences']")

Session created: 06cf9cc4...
Preferences stored in state['user:preferences']


## Execute Workflow

Run the multi-agent workflow to create the travel itinerary:

In [13]:
print("="*70)
print("EXECUTING WORKFLOW")
print("="*70)
print(f"Book: {BOOK_TITLE}")
print(f"Author: {AUTHOR or 'Unknown'}\n")

# Create the prompt
prompt = f"""Create a literary travel itinerary for "{BOOK_TITLE}" by {AUTHOR or 'unknown author'}.

Execute all steps and return the complete combined results."""

user_message = types.Content(
    role='user',
    parts=[types.Part(text=prompt)]
)

print(f"USER REQUEST:")
print(f"{prompt}\n")
print("Starting workflow execution...\n")
print("="*70)

# Run workflow and collect results
final_response = None
event_count = 0

async for event in runner.run_async(
    user_id=USER_ID,
    session_id=session_id,
    new_message=user_message
):
    event_count += 1
    
    # Show progress
    if event.author:
        print(f"[{event_count}] {event.author}")
    
    if event.is_final_response():
        final_response = event
        print(f"\n✅ Workflow complete ({event_count} events)")

print("\n" + "="*70)

2025-11-22 14:40:44,378 - INFO - google_adk.google.adk.models.google_llm - Sending out request, model: gemini-2.0-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False


EXECUTING WORKFLOW
Book: gone with the wind
Author: Unknown

USER REQUEST:
Create a literary travel itinerary for "gone with the wind" by unknown author.

Execute all steps and return the complete combined results.

Starting workflow execution...



2025-11-22 14:40:45,034 - INFO - httpx - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash-lite:generateContent "HTTP/1.1 200 OK"
2025-11-22 14:40:45,039 - INFO - google_adk.google.adk.models.google_llm - Response received from the model.
2025-11-22 14:40:45,041 - INFO - google_adk.google.adk.models.google_llm - Sending out request, model: gemini-2.0-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False
2025-11-22 14:40:45,042 - INFO - google_genai.models - AFC is enabled with max remote calls: 10.


[1] book_metadata_researcher

✅ Workflow complete (1 events)


2025-11-22 14:40:46,103 - INFO - httpx - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash-lite:generateContent "HTTP/1.1 200 OK"
2025-11-22 14:40:46,109 - INFO - google_adk.google.adk.models.google_llm - Response received from the model.
2025-11-22 14:40:46,112 - INFO - google_adk.google.adk.models.google_llm - Sending out request, model: gemini-2.0-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False
2025-11-22 14:40:46,114 - INFO - google_genai.models - AFC is enabled with max remote calls: 10.


...

[12] city_researcher

✅ Workflow complete (12 events)


2025-11-22 14:41:04,534 - INFO - httpx - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash-lite:generateContent "HTTP/1.1 200 OK"
2025-11-22 14:41:04,538 - INFO - google_adk.google.adk.models.google_llm - Response received from the model.
2025-11-22 14:41:04,543 - INFO - google_adk.google.adk.models.google_llm - Sending out request, model: gemini-2.0-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False
2025-11-22 14:41:04,544 - INFO - google_genai.models - AFC is enabled with max remote calls: 10.


[13] city_formatter

✅ Workflow complete (13 events)


2025-11-22 14:41:10,205 - INFO - httpx - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash-lite:generateContent "HTTP/1.1 200 OK"
2025-11-22 14:41:10,210 - INFO - google_adk.google.adk.models.google_llm - Response received from the model.


[14] trip_composer

✅ Workflow complete (14 events)



## Extract and Display Results

In [14]:
# Extract result from final response
result_data = {}
if final_response and final_response.content and final_response.content.parts:
    for part in final_response.content.parts:
        if hasattr(part, 'text') and part.text:
            json_start = part.text.find('{')
            json_end = part.text.rfind('}') + 1
            
            if json_start >= 0 and json_end > json_start:
                try:
                    result_data = json.loads(part.text[json_start:json_end])
                    break
                except json.JSONDecodeError as e:
                    logger.error(f"Failed to parse JSON: {e}")

# Display the itinerary
cities_list = result_data.get('cities', [])

if cities_list:
    print(f"\nTRAVEL ITINERARY - {len(cities_list)} Cities")
    print("="*70)
    
    for i, city_plan in enumerate(cities_list, 1):
        print(f"\nCity #{i}: {city_plan.get('name')}, {city_plan.get('country')}")
        print(f"Days suggested: {city_plan.get('days_suggested', 1)}")
        
        if city_plan.get('overview'):
            print(f"\nOverview: {city_plan.get('overview')}")
        
        stops = city_plan.get('stops', [])
        if stops:
            print(f"\nStops ({len(stops)}):")
            for j, stop in enumerate(stops, 1):
                print(f"  {j}. {stop.get('name')} ({stop.get('type')})")
                print(f"     {stop.get('reason')}")
    
    # Summary
    summary_text = result_data.get('summary_text')
    if summary_text:
        print("\n" + "="*70)
        print("TRIP SUMMARY")
        print("="*70)
        print(f"\n{summary_text}\n")
else:
    print("\n❌ No itinerary data found")


TRAVEL ITINERARY - 3 Cities

City #1: Atlanta, USA
Days suggested: 2

Overview: Immerse yourself in the heart of the Gone With The Wind story in Atlanta, the city that serves as a pivotal backdrop to the novel. Explore the places that shaped Margaret Mitchell's life and the Civil War-era atmosphere.

Stops (4):
  1. Margaret Mitchell House and Museum (museum)
     This is where Margaret Mitchell penned much of the novel. The museum offers an intimate look into her life and the creation of the epic.
  2. Oakland Cemetery (cemetery)
     Pay your respects at the final resting place of Margaret Mitchell, gaining a moment of reflection.
  3. Mary Mac's Tea Room (restaurant)
     Enjoy a meal at this iconic Southern restaurant, savoring classic dishes amidst historic ambiance.
  4. Atlanta History Center (museum)
     Explore the Civil War exhibit, to understand the historical context.

City #2: Clayton County, USA
Days suggested: 1

Overview: Venture into Clayton County, the landscape tha

## Context Statistics

Check the session context and token usage:

In [15]:
# Get the session to check context
session = await session_service.get_session(
    app_name="storyland",
    user_id=USER_ID,
    session_id=session_id
)

# Get context statistics
stats = context_manager.get_context_stats(session.events)

print("Context Statistics")
print("="*40)
print(f"Total events: {stats['num_events']}")
print(f"Estimated tokens: {stats['estimated_tokens']}")
print(f"Within limit: {stats['within_limit']}")
print(f"Should compact: {context_manager.should_compact(session.events)}")

# Show preferences from state
prefs = session.state.get("user:preferences", {})
print(f"\nUser Preferences in State:")
for key, value in prefs.items():
    print(f"  {key}: {value}")

Context Statistics
Total events: 15
Estimated tokens: 4174
Within limit: True
Should compact: False

User Preferences in State:
  budget: moderate
  preferred_pace: moderate
  prefers_museums: True
  travels_with_kids: False


## Summary

This notebook demonstrates the modular architecture:

- **Models** ([models/](models/)) - Pydantic data structures
- **Tools** ([tools/](tools/)) - External API integrations (Google Books, preferences)
- **Agents** ([agents/](agents/)) - Specialized AI agents
- **Services** ([services/](services/)) - Session and context management
- **Config** ([common/config.py](common/config.py)) - Centralized configuration

### How Preferences Work

1. **Set preferences** in `USER_PREFERENCES` dict above
2. **Stored in session state** as `user:preferences`
3. **`reader_profile_agent`** calls `get_preferences_tool` to read from `ToolContext.state`
4. **`trip_composer`** sees the preferences summary and personalizes the itinerary

### Next Steps

1. **Try the CLI**: `python main.py "your book title" --budget luxury --pace relaxed`
2. **Enable persistence**: Set `USE_DATABASE=True` above
3. **Customize preferences**: Edit `USER_PREFERENCES` dict
4. **Add new agents**: Edit files in [agents/](agents/) directory
5. **Add new tools**: Create new tools in [tools/](tools/) directory